In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class LabelBinarizerEx(BaseEstimator,TransformerMixin):
    '''
    extend for sklearn label binarizer
    take account for missing values
    '''
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        '''
        @X dataFrame with one column
        '''
        df=pd.get_dummies(X)
        self.classes=[c[c.rindex('_')+1:] for c in df.columns]
        self.columns=list(df.columns)
        return self
    
    def transform(self,X,y=None):
        all_zero=[0 for c in self.classes]
        values=np.array([all_zero for i in range(len(X))])
        for i in range(len(X)):
            j=np.searchsorted(self.classes, X.iloc[i][0])
            if j<len(self.classes):
                values[i][j]=1
        return values
    
    def fit_transform(self,X,y=None):
        self.fit(X)
        return self.transform(X)
    
import unittest as ut

from sklearn.pipeline import Pipeline

class Test(ut.TestCase):
    def testLabelize(self):
        df=pd.DataFrame({'sex':['m','f',np.nan]})
        
        l=LabelBinarizerEx()
        l.fit(df)
        self.assertEqual(['f','m'],l.classes)
        self.assertEqual(['sex_f','sex_m'],l.columns)
        
        data=l.transform(pd.DataFrame({'sex':['m','f',np.nan]}))  
        self.assertTrue(np.array_equal(np.array([[0,1],[1,0],[0,0]]), data))
        
    def testNumCate(self):
        df=pd.DataFrame({'id':[1,2,1]})
        
        l=LabelBinarizerEx()
        l.fit(df)
        self.assertEqual(['1','2'],l.classes)
        self.assertEqual(['id_1','id_2'],l.columns)
                
if __name__ == '__main__':
    ut.main(argv=['ignored', '-v'], exit=False)

testLabelize (__main__.Test) ... ok
testNumCate (__main__.Test) ... ERROR

ERROR: testNumCate (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-8d3129d3c79e>", line 55, in testNumCate
    l.fit(df)
  File "<ipython-input-1-8d3129d3c79e>", line 18, in fit
    self.classes=[c[c.rindex('_')+1:] for c in df.columns]
  File "<ipython-input-1-8d3129d3c79e>", line 18, in <listcomp>
    self.classes=[c[c.rindex('_')+1:] for c in df.columns]
ValueError: substring not found

----------------------------------------------------------------------
Ran 2 tests in 0.016s

FAILED (errors=1)


In [16]:
df=pd.DataFrame({'id':[1,2,1]})
df['id']=df['id'].astype(str)
df
df.info()
pd.get_dummies(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
id    3 non-null object
dtypes: object(1)
memory usage: 104.0+ bytes


,id_1,id_2
0,1,0
1,0,1
2,1,0
